In [1]:
from mlflow.tracking import MlflowClient
import mlflow


# MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
mlflow.set_tracking_uri("http://127.0.0.1:8080")

In [2]:
MLFLOW_TRACKING_URI = "http://127.0.0.1:8080"

In [3]:
# client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client = MlflowClient()

client.search_experiments()

[<Experiment: artifact_location='file:///c:/Users/artkr/OneDrive/เอกสาร/GitHub/cpe393-mlflow/mlruns/1', creation_time=1746534332069, experiment_id='1', last_update_time=1746534332069, lifecycle_stage='active', name='mlops_nyc_taxi', tags={}>,
 <Experiment: artifact_location='file:///c:/Users/artkr/OneDrive/เอกสาร/GitHub/cpe393-mlflow/mlruns/0', creation_time=1746534332061, experiment_id='0', last_update_time=1746534332061, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
client.create_experiment(name="my-new-experiment")

'2'

In [5]:
from mlflow.entities import ViewType

# Replace '1' with your actual experiment ID if different
runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 100",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")


run id: 2a2829c2140c4e5986864f150d651e5d, rmse: 29.2233
run id: 55fe47924956473ab5379c732c40dd7c, rmse: 33.9278
run id: 3f3fb49315de45838ee7f4d732224c38, rmse: 58.6168
run id: d962b6ca3381496d8f6d9ed12198bd26, rmse: 73.0615


In [6]:
import pickle

with open("preprocessor.b", "rb") as f:
    dv = pickle.load(f)


In [8]:
run_id = "55fe47924956473ab5379c732c40dd7c"
model_uri = f"runs:/{run_id}/models_mlflow"

result = mlflow.register_model(
    model_uri=model_uri,
    name="duration-prediction"
)


Registered model 'duration-prediction' already exists. Creating a new version of this model...
2025/05/06 19:40:33 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: duration-prediction, version 2
Created version '2' of model 'duration-prediction'.


In [11]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

models = client.search_registered_models()
for model in models:
    print(f"Model name: {model.name}")
    for v in model.latest_versions:
        print(f"  Version: {v.version}, Stage: {v.current_stage}, Run ID: {v.run_id}")


Model name: duration-prediction
  Version: 2, Stage: None, Run ID: 55fe47924956473ab5379c732c40dd7c


In [12]:
import urllib.request

urls = [
    "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-03.parquet",
]

for url in urls:
    filename = url.split("/")[-1]
    print(f"Downloading {filename}...")
    urllib.request.urlretrieve(url, filename)
    print("Done.")


Done.


In [13]:
# Promote version 1 to Staging
client.transition_model_version_stage(
    name="duration-prediction",
    version="1",
    stage="Staging"
)

# Promote version 2 to Production
client.transition_model_version_stage(
    name="duration-prediction",
    version="2",
    stage="Production"
)


C:\Users\artkr\AppData\Local\Temp\ipykernel_18292\547606745.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(
C:\Users\artkr\AppData\Local\Temp\ipykernel_18292\547606745.py:9: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1746535233591, current_stage='Production', description='', last_updated_timestamp=1746535246354, name='duration-prediction', run_id='55fe47924956473ab5379c732c40dd7c', run_link='', source='file:///c:/Users/artkr/OneDrive/เอกสาร/GitHub/cpe393-mlflow/mlruns/1/55fe47924956473ab5379c732c40dd7c/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id='', version='2'>

In [ ]:
# import pandas as pd

# df_test = pd.read_parquet("green_tripdata_2021-03.parquet")


In [ ]:
# # Add this before dropping datetime columns
# df_test["duration"] = df_test.lpep_dropoff_datetime - df_test.lpep_pickup_datetime
# df_test["duration"] = df_test["duration"].apply(lambda td: td.total_seconds() / 60)
# df_test = df_test[(df_test.duration >= 1) & (df_test.duration <= 60)].copy()


In [ ]:
# df_test["pickup_hour"] = df_test["lpep_pickup_datetime"].dt.hour
# df_test["pickup_dayofweek"] = df_test["lpep_pickup_datetime"].dt.dayofweek
# df_test["dropoff_hour"] = df_test["lpep_dropoff_datetime"].dt.hour
# df_test["dropoff_dayofweek"] = df_test["lpep_dropoff_datetime"].dt.dayofweek

# # Drop raw datetime columns if they weren’t used in training
# df_test = df_test.drop(columns=["lpep_pickup_datetime", "lpep_dropoff_datetime"])


In [ ]:
# X_test = dv.transform(df_test.to_dict(orient="records"))


In [ ]:
# model_name = "duration-prediction"
# latest_versions = client.get_latest_versions(name=model_name)
# for mv in latest_versions:
#     print(f"Model {model_name}, version {mv.version}: stage={mv.current_stage}, run_id={mv.run_id}")


Model duration-prediction, version 1: stage=Staging, run_id=e76d234c62bd4648b63444f033e760a8
Model duration-prediction, version 2: stage=Production, run_id=e76d234c62bd4648b63444f033e760a8


C:\Users\artkr\AppData\Local\Temp\ipykernel_6092\1135971749.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [14]:
import mlflow.pyfunc

staging_model = mlflow.pyfunc.load_model("models:/duration-prediction/Staging")
production_model = mlflow.pyfunc.load_model("models:/duration-prediction/Production")


In [15]:
import pandas as pd

df_test = pd.read_parquet("green_tripdata_2021-03.parquet")

In [16]:
df_test["duration"] = df_test.lpep_dropoff_datetime - df_test.lpep_pickup_datetime
df_test["duration"] = df_test["duration"].apply(lambda td: td.total_seconds() / 60)

In [17]:
df_test = df_test[(df_test.duration >= 1) & (df_test.duration <= 60)].copy()

In [18]:
df_test["PULocationID"] = df_test["PULocationID"].astype(str)
df_test["DOLocationID"] = df_test["DOLocationID"].astype(str)
df_test["PU_DO"] = df_test["PULocationID"] + "_" + df_test["DOLocationID"]

In [19]:
categorical = ["PU_DO"]
numerical = ["trip_distance"]
test_dicts = df_test[categorical + numerical].to_dict(orient="records")

In [20]:
X_test = dv.transform(test_dicts)

In [21]:
y_test = df_test["duration"].values

In [22]:
y_pred_staging = staging_model.predict(X_test)
y_pred_production = production_model.predict(X_test)

# Example metric: MAE
from sklearn.metrics import mean_absolute_error

mae_staging = mean_absolute_error(df_test["duration"], y_pred_staging)
mae_production = mean_absolute_error(df_test["duration"], y_pred_production)

print(f"MAE Staging: {mae_staging}")
print(f"MAE Production: {mae_production}")


MAE Staging: 5.365534611422979
MAE Production: 5.365534611422979


In [23]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

# Get version of current Staging model
staging_version = client.get_latest_versions("duration-prediction", stages=["Staging"])[0].version

# Promote to Production
client.transition_model_version_stage(
    name="duration-prediction",
    version=staging_version,
    stage="Production",
    archive_existing_versions=True
)


C:\Users\artkr\AppData\Local\Temp\ipykernel_18292\3252090468.py:6: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  staging_version = client.get_latest_versions("duration-prediction", stages=["Staging"])[0].version
C:\Users\artkr\AppData\Local\Temp\ipykernel_18292\3252090468.py:9: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1746535229118, current_stage='Production', description='', last_updated_timestamp=1746535270651, name='duration-prediction', run_id='55fe47924956473ab5379c732c40dd7c', run_link='', source='file:///c:/Users/artkr/OneDrive/เอกสาร/GitHub/cpe393-mlflow/mlruns/1/55fe47924956473ab5379c732c40dd7c/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id='', version='1'>

.....

In [24]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 100",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [38]:
runs

[<Run: data=<RunData: metrics={'rmse': 29.223338109056154}, params={'learning_rate': '0.47390461808219303',
  'max_depth': '40',
  'min_child_weight': '1.7591586781822681',
  'objective': 'reg:linear',
  'reg_alpha': '0.05263667882154712',
  'reg_lambda': '0.03191746437453483',
  'seed': '42'}, tags={'mlflow.runName': 'persistent-sow-972',
  'mlflow.source.name': 'C:\\Users\\artkr\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\ipykernel_launcher.py',
  'mlflow.source.type': 'LOCAL',
  'mlflow.user': 'artkr',
  'model': 'xgboost'}>, info=<RunInfo: artifact_uri='file:///c:/Users/artkr/OneDrive/เอกสาร/GitHub/cpe393-mlflow/mlruns/1/2a2829c2140c4e5986864f150d651e5d/artifacts', end_time=1746534858782, experiment_id='1', lifecycle_stage='active', run_id='2a2829c2140c4e5986864f150d651e5d', run_name='persistent-sow-972', run_uuid='2a2829c2140c4e5986864f150d651e5d', start_time=1746534833794, status='FINISHED', 

In [25]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [55]:
run_id = "5f3efbc386f340c2ad647a6b9ee5fa5f"  #insert your run id
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2025/05/06 20:08:11 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 3
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1746536891765, current_stage='None', description='', last_updated_timestamp=1746536891765, name='nyc-taxi-regressor', run_id='5f3efbc386f340c2ad647a6b9ee5fa5f', run_link='', source='file:///c:/Users/artkr/OneDrive/เอกสาร/GitHub/cpe393-mlflow/mlruns/1/5f3efbc386f340c2ad647a6b9ee5fa5f/artifacts/model', status='READY', status_message=None, tags={}, user_id='', version='3'>

In [27]:
run_id = "55fe47924956473ab5379c732c40dd7c"  #insert your run id
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2025/05/06 19:44:24 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 1
Created version '1' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1746535464934, current_stage='None', description='', last_updated_timestamp=1746535464934, name='nyc-taxi-regressor', run_id='55fe47924956473ab5379c732c40dd7c', run_link='', source='file:///c:/Users/artkr/OneDrive/เอกสาร/GitHub/cpe393-mlflow/mlruns/1/55fe47924956473ab5379c732c40dd7c/artifacts/model', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [60]:
# Promote version 1 to Staging
client.transition_model_version_stage(
    name="nyc-taxi-regressor",
    version="1",
    stage="Staging"
)

# Promote version 2 to Production
client.transition_model_version_stage(
    name="nyc-taxi-regressor",
    version="3",
    stage="Production"
)

# Promote version 2 to Production
client.transition_model_version_stage(
    name="nyc-taxi-regressor",
    version="2",
    stage="Archived"
)



C:\Users\artkr\AppData\Local\Temp\ipykernel_18292\2869865357.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(
C:\Users\artkr\AppData\Local\Temp\ipykernel_18292\2869865357.py:9: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(
C:\Users\artkr\AppData\Local\Temp\ipykernel_18292\2869865357.py:16: FutureWarning: ``mlflow.tracki

<ModelVersion: aliases=[], creation_timestamp=1746535840404, current_stage='Archived', description='', last_updated_timestamp=1746536986886, name='nyc-taxi-regressor', run_id='2a2829c2140c4e5986864f150d651e5d', run_link='', source='file:///c:/Users/artkr/OneDrive/เอกสาร/GitHub/cpe393-mlflow/mlruns/1/2a2829c2140c4e5986864f150d651e5d/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id='', version='2'>

In [61]:
#check model version and stages
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Staging
version: 2, stage: Archived
version: 3, stage: Production


C:\Users\artkr\AppData\Local\Temp\ipykernel_18292\1279244624.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


Comparing versions and selecting the new "Production" model
In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
Download the DictVectorizer that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
Preprocess the test set using the DictVectorizer so we can properly feed the regressors.
Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
Based on the results, update the "Production" model version accordingly.
